## 准备数据

In [4]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    x = x.reshape(x.shape[0], 28, 28,1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28,1)
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).batch(100)
    
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(prepare_mnist_features_and_labels)
    test_ds = test_ds.take(20000).batch(20000)
    return ds, test_ds

def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

## 建立模型

In [5]:
class myConvModel(keras.Model):
    def __init__(self):
        super(myConvModel, self).__init__()
        self.l1_conv = Conv2D(filters=32, 
                              kernel_size=(5, 5), 
                              activation='relu', padding='same')
        
        self.l2_conv = Conv2D(filters=64, 
                              kernel_size=(5, 5), 
                              activation='relu',padding='same')
        
        self.pool = MaxPooling2D(pool_size=(2, 2), strides=2)
        
        self.flat = Flatten()
        self.dense1 = layers.Dense(100, activation='tanh')
        self.dense2 = layers.Dense(10)
    @tf.function
    def call(self, x):
        '''shape(x) = [32, 28, 28, 1]'''
        h1 = self.l1_conv(x) # [32, 28, 28, 32]
        h1_pool = self.pool(h1) # [32, 14, 14, 32]
        h2 = self.l2_conv(h1_pool) #[32, 14, 14, 64]
        '''kernel 参数量64*5*5*32'''
        h2_pool = self.pool(h2) #[32, 7, 7, 64]
        flat_h = self.flat(h2_pool) #[32, 7*7*64]
        dense1 = self.dense1(flat_h) #[32, 100]
        logits = self.dense2(dense1) #[32, 10]
        probs = tf.nn.softmax(logits, axis=-1) #[32, 10]
        return probs

model = myConvModel()
optimizer = optimizers.Adam()

## 编译， fit以及evaluate

In [6]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
train_ds, test_ds = mnist_dataset()
model.fit(train_ds, epochs=5)
model.evaluate(test_ds)

W0907 16:16:17.882842 4609922496 training_utils.py:1436] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/5
200/200 [==============================] - 30s 149ms/step - loss: 0.3044 - accuracy: 0.8122
Epoch 2/5
200/200 [==============================] - 29s 147ms/step - loss: 0.0852 - accuracy: 0.9713
Epoch 3/5
200/200 [==============================] - 29s 144ms/step - loss: 0.0504 - accuracy: 0.9838
Epoch 4/5
200/200 [==============================] - 27s 135ms/step - loss: 0.0301 - accuracy: 0.9901
Epoch 5/5
1/1 [==============================] - 6s 6s/step - loss: 0.0502 - accuracy: 0.9834


[0.050157830119132996, 0.9834]

In [ ]:
f(1) == 1
f(2) == 2

In [ ]:
f(2) == 5
f(1) == 2